# Functional Connectivity

In this tutorial, we'll cover the basics of functional connectivity analyses. In brief, functional connectivity examines the correlations between signals of brain regions. Functional connectivity is often analyzed in the context of resting-state fMRI, which often referred to as 'intrinsic' functional connectivity because these correlations arise in the absence of tasks and behaviour.

We'll analyze a 20 minute resting-state scan of one subject to explore the methodology and types of analyses we can perform. We'll start by getting an _atlas_ that let's us define non-overlapping cortical regions. Then, we'll extract the _mean timeseries_ of these regions using nilearn's `NiftiLabelsMasker`.  `NiftiLabelsMasker` is similar to previous masker objects we've used, except that it computes the mean timeseries for our regions defined according to some atlas/parcellation. This object is incredibly convenient. 

Next, we'll run a functional connectivity analysis by correlating the timeseries between each possible pairs of regions, examine the resulting connectivity matrix, and discuss how we can begin to analyze this matrix.

To do connectivity analysis, we'll need to expand the imports to include some key packages and classes/functions.

For one, we'll use `bct`, which is a Python version of the Brain Connectivity Toolbox. This was [originally in MATLAB](https://sites.google.com/site/bctnet/). The Python package (see the [github wiki](https://github.com/aestrivex/bctpy/wiki)) is excellent and contains many useful functions for the remaining lessons.

In addition to `bct`, we'll also import a few functions/classes from nilearn that will let us first define our regions (`fetch_atlas_schaefer_2018`) and then extract the mean timeseries of these regions (`NiftiLabelsMasker`). We'll also import  `ConnectivityMeasure` from nilearn, which makes it really easy to run a functional connectivity analysis. Finally, we'll import the plotting and image modules entirely rather than their individual functions. 

In [40]:
import numpy as np
import pandas as pd
from scipy import stats, special
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib

from nilearn.datasets import fetch_atlas_schaefer_2018
from nilearn.maskers import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn import plotting, image
import bct

%matplotlib inline

## 1. Getting an atlas/parcellation

Atlases (often also called parcellations) provide a map of brain regions. Atlases can come in a variety of formats, including a labelled 3D NIfTI image (i.e. each voxel labelled according to its region), a 4D NIfTI image in which each volume is a probabilistic map of a region (each voxel is assigned a probability that it belongs to each region), or a list of coordinates that denote the center of each region. By far, the most common format is the labelled 3D image, which we'll use here.

Nilearn has a [number of functions that let us fetch popular atlases](https://nilearn.github.io/stable/modules/datasets.html#atlases). We'll use one of these functions to fetch the Schaefer altas ([link to the paper](https://academic.oup.com/cercor/article/28/9/3095/3978804)), which is a recent atlas that determines region boundaries based on sudden changes in voxelwise functional connectivity (called 'gradients') and clustering approaches.

In [ ]:
atlas = fetch_atlas_schaefer_2018(n_rois=200, resolution_mm=2)

# we can get the image out of the dictionary
atlas_img = nib.load(atlas['maps'])

We can plot the atlas overlaid on top of a MNI template. The coordinates I've selected highlight some nice features of the atlas:

In [ ]:
plotting.plot_roi(atlas_img, cmap='jet', cut_coords=[6, 20, 50])

As you can see, each region is shown as a different colour, meaning that voxels all belonging to the same region have the same numerical value and therefore same colour. Think of regions as just a group of contiguous voxels, all of which have the same value in this atlas.

Alternatively, you can explore the atlas interactively:

In [ ]:
plotting.view_img(atlas_img, cmap='jet', symmetric_cmap=False)

As mentioned above, each voxel has a numerical label in the image, which corresponds to the region it belongs to. In other words, "region 1" is populated by voxels all with the value `1`. But what _is_ "region 1"? Well, atlases often come with region labels or names. We can get these names with the labels using the code below, which are shown in ascending numerical order. We see that "region 1" is actually called "7Networks_LH_Vis_1", "region 2" is "7Networks_LH_Vis_2", and so forth.

In [ ]:
# decode just converts it from byte-format to string
labels = [x.decode() for x in atlas['labels']]
labels

## 2. Extracting regions timeseries from resting state

Now that we have an atlas, we want to extract the mean timeseries from each region--i.e. the time-varying average BOLD signal of all voxels in the region. We also want to apply some post-processing to these timeseries so that we can remove low-frequency trends in the data (temporal filtering) and regress out sources of noise (i.e. confound regression). This may seem daunting because our example atlas has 200 regions, but in actuality this is a very simple task to do in nilearn.

First, we'll load in our confounds that we wish to regress out. Remember that confounds are found in `regressor_confounds.tsv` files produced by fmriprep. Instead of loading in the whole file, we'll only load in the confounds that we want. You can see that I have specified more confounds than we are used to seeing (only the 6 motion parameters). We are going to get the 6 motion parameters, their derivatives, their square, and the derivatives of the squares (24 motion parameters total). We'll also include the framewise displacement (a composite measure of motion), and the mean signals of white matter and CSF. The two latter confounds give us signal fluctuations that we presume to be physiological noise that we want to remove from our data.

Try also adding 'global_signal' by uncommenting the final line in the cell below:

In [18]:
confound_names = [
    'trans_x', 
    'trans_y', 
    'trans_z',
    'trans_x_derivative1', 
    'trans_y_derivative1', 
    'trans_z_derivative1',
    'trans_x_power2', 
    'trans_y_power2', 
    'trans_z_power2',
    'trans_x_derivative1_power2', 
    'trans_y_derivative1_power2', 
    'trans_z_derivative1_power2',
    'rot_x', 
    'rot_y', 
    'rot_z', 
    'rot_x_derivative1',
    'rot_y_derivative1', 
    'rot_z_derivative1',
    'rot_x_power2',
    'rot_y_power2', 
    'rot_z_power2',
    'rot_x_derivative1_power2',
    'rot_y_derivative1_power2',
    'rot_z_derivative1_power2', 
    'framewise_displacement', 
    'csf', 
    'white_matter'
]

# confound_names.append('global_signal')

In [ ]:
confounds = pd.read_table('rs-data/sample_resting_state_confounds_regressors.tsv', usecols=confound_names)
confounds.head()

Note that the first row has not-a-number (`NaN`) for the derivatives; this is unavoidable because derivatives are the rate of change between data points, so it requires two datapoints by definition. Nilearn does not accept `NaN`s so we need to drop that row. This row corresponds to the first functional volume, which now needs to be dropped as well.

Now let's load in our functional data, and then drop the first volume:

In [ ]:
# load image and show the number of volumes
func_img = nib.load('rs-data/sample_resting_state.nii.gz')
print("Total number of volumes:", func_img.shape[-1])

# remove the first volume by selecting all volumes except the first
func_img = image.index_img(func_img, slice(1, None))
print("Total number of volumes after removal:", func_img.shape[-1])

And let's not forget to drop the first row of our confounds:

In [ ]:
# convert confounds to numpy array (from DataFrame), and drop first row
confounds = confounds.values[1:, :]
print("Total number of rows:", confounds.shape[0])

The number of imaging volumes and confounds rows are now the same, and we are ready to extract data. We will use `NiftiLabelsMasker` post-process, apply confound regression, and extract the mean timeseries of each region. The extracted data is a matrix of region timeseries, i.e. a time by region array:

In [24]:
# set up our parameters
masker = NiftiLabelsMasker(labels_img=atlas_img, high_pass=.01, detrend=True, standardize=True, t_r=2)
# extract
data = masker.fit_transform(func_img, confounds=confounds)

print("Time by region array:", data.shape)

We can visualize this array to get a birds-eye view of the data. In doing so, we'll be able to see the signal fluctuations of each region (columns) across time (rows). By simply examining this data, you can get a nice intuition of which regions covary, and the nature of their covariation.

In [ ]:
plt.figure(figsize=(5, 12))
plt.imshow(data, aspect='auto')

## 3. Computing functional connectivity

### 3.1 Example correlation

Functional connectivity is essentially computing correlations between each column in the matrix. For example:

In [ ]:
# get two regions and compute correlation
region1 = data[:, 0]
region2 = data[:, 1]
r, p = stats.pearsonr(region1, region2)

# multipanel figure (axes is now an array)
fig, axes = plt.subplots(ncols=2, figsize=(12, 3))
# unpack axes
ax1, ax2 = axes
# timeseries plot
ax1.plot(range(data.shape[0]), region1) # blue
ax1.plot(range(data.shape[0]), region2) # orange
ax1.set(xlabel='Volume', ylabel='Signal (z)', title='Timeseries')
# correlation plot
ax2.scatter(region1, region2, alpha=.5, c='C7')
ax2.set(xlabel='Region 1 (z)', ylabel='Region 2 (z)', title=f'r = {r:.3f}');

On the left, we see the timeseries for both regions. The right shows the scatter plot of both region, where each datapoint is a volume/timepoint. Signal increases in region 1 tends to coincide with signal increases in region 2.

### 3.2 Correlation or connectivity Matrix

We can easily expand this to all possible combinations of correlations in our data by using nilearn's `ConnectivityMeasure` ([see documentation](https://nilearn.github.io/stable/modules/generated/nilearn.connectome.ConnectivityMeasure.html#nilearn.connectome.ConnectivityMeasure)). Calling the `fit_transform` method returns a _connectivity matrix_ showing each pairwise correlation. 

In [ ]:
connect = ConnectivityMeasure(kind='correlation')

# analyze our matrix as a list because ConnectivityMeasure expects multi-subject data
cmat = connect.fit_transform([data])
# get the connectivity matrix out of the list
cmat = cmat[0]

# show the shape
cmat.shape

Now we can plot this matrix:

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(cmat, labels=labels, figure=fig, vmin=-1, vmax=1)

We can see that this is a 200x200 connectivity matrix, and the overwhelming majority of correlations are positive. Let's take a look at a submatrix to break this large matrix down and understand how to interpret it. We'll take the first 9 regions, which correspond to the left visual network:

In [ ]:
left_vis = cmat[:9, :9]

fig, ax = plt.subplots(figsize=(4, 4))
plotting.plot_matrix(left_vis, labels=labels[:9], figure=fig, vmin=-1, vmax=1)

Clearly all of these regions are visual areas, indicated by the 'Vis' in their names. Where are these in the brain? Let's check:

In [ ]:
vis_img = image.math_img("np.where(np.isin(img, np.arange(1, 10)), img, 0)", img=atlas_img)
plotting.plot_roi(vis_img, vmax=10, vmin=1, cmap='tab10', colorbar=True,
                  cut_coords=[0, -10, -20, -30, -40, -50], display_mode='x')

In the connectivity matrix above, the order of regions is simply the order in which they appear in the atlas. As evident, regions all belonging to the same network and hemisphere, like the left Visual network, are grouped together. We can see the cross-hemisphere connectivity by looking at the clusters in the off-diagonal elements that also mimic the diagonal.

We can re-plot the connectivity matrix in a way that groups regions together by how they're connected, instead of their order in the atlas. Doing so will give you a sense of what are highly connected all together, regardless of their respective hemisphere and network.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(cmat, labels=labels, figure=fig, vmin=-1, vmax=1, reorder=True)

## 4. Thresholding

Often we want retain correlations that fall above a certain threshold. We can threshold matrices in a variety of different ways. Of course, you may choose to convert your _r_ values to _p_ values, and then apply statistical thresholding (e.g., _p_ < .05 or _p_ < .01) and then do some sort of multiple comparisons correction (e.g., false-discovery rate correction).

### 4.1 Absolute thresholding

A more conventional and simpler approach is to threshold at _r_ = .3. This has become a bit of an arbitrary convention in the field, but it is sensible because _r_ values above .3 are generally considered to be moderate effect size. In our case here, _r_ > .3 corresponds to $p < 10^{-14}$ (where _n_ = 611, the number of volumes/timepoints). To do this, we can use `threshold_absolute` from `bctpy`:  

In [ ]:
threshold = .3
thresh_mat = bct.threshold_absolute(cmat, threshold)

fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(thresh_mat, labels=atlas['labels'], figure=fig, vmin=-1, vmax=1)

### 4.2 Proportional thresholding

We can also threshold based on the top proportion of correlations. For instance, we can select the correlations that fall within the 90th percentile (i.e. top 10%). We can use the `threshold_proportional` function from `bctpy`:

In [ ]:
threshold = .1 # top 10%
thresh_mat = bct.threshold_proportional(cmat, threshold)

fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(thresh_mat, labels=atlas['labels'], figure=fig, vmin=-1, vmax=1)

This is much sparser, which may be useful for certain applications.

### 4.3 Binarizing a thresholded matrix

We can also binarize our connectivity matrix (convert every nonzero value to 1). Binarization is necessary for some graph theoretical methods that we'll explore in the next tutorial. We can call the `binarize` function on one of our thresholded matrices.

In [ ]:
binary_mat = bct.binarize(thresh_mat)

fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(binary_mat, labels=atlas['labels'], figure=fig, cmap='binary', colorbar=False)

## 5. Saving a connectivity matrix

Now we'll just save our original, unthresholded connectivity matrix to use in the next tutorial.

In [39]:
np.savetxt("connectivity.csv", cmat, delimiter=",")